In [1]:

import os
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
from dotenv import load_dotenv
import bs4
from langchain_groq import ChatGroq

load_dotenv()

## load the Groq API key
groq_api_key = os.environ['GROQ_API_KEY']

In [33]:

embeddings = OllamaEmbeddings()
loader=WebBaseLoader("https://www.taniarascia.com/getting-started-with-react/")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=250)
final_documents = text_splitter.split_documents(docs)
vectors = FAISS.from_documents(final_documents, embeddings)

In [34]:

llm=ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")



prompt=ChatPromptTemplate.from_template(
"""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)



In [37]:

response=retrieval_chain.invoke({"input":'create 5 random objective type questions which have( 4 options out of 1 is correct) from the context given.The questions should be on methods and functions in react.  give its answer also. the questions should be based on react programming not on its general awareness. questions should not be on its features, pre requisites, compilers, comparisons with other languages, authors  '})
print(response['answer'])

1. What is the purpose of the `componentDidMount()` method in the context of React?
a) It is used to render a component
b) It is used to update a component
c) It is used to handle events
d) It is used to run code after a component is mounted to the DOM

Answer: d) It is used to run code after a component is mounted to the DOM

2. Which of the following is used to pass data from a parent component to a child component in React?
a) state
b) props
c) refs
d) keys

Answer: b) props

3. What is the role of the `key` prop in the following code snippet?
```php
<tr key={index}>
  ...
</tr>
```
a) It is used to identify the row uniquely
b) It is used to style the row
c) It is used to handle events on the row
d) It is used to pass data to the row

Answer: a) It is used to identify the row uniquely

4. Which of the following is used to remove an item from an array in the `removeCharacter()` method?
a) filter()
b) map()
c) reduce()
d) forEach()

Answer: a) filter()

5. Which of the following is us